In [9]:
import glob
from osmread import parse_file, Way,Node,Relation
import shapely.geometry as shpgeo
import overpy
from shapely.ops import linemerge
import time

In [2]:
api = overpy.Overpass()
shpgeo.Polygon.centroid

In [11]:
def lonlat_in_way(way):
    coors = []
    for node in way.nodes:
        coors.append((node.lon, node.lat))
    return coors
def is_closed_way(way):
    return way.nodes[0] == way.nodes[-1]

def get_relation(relation,need_api=True):
    while True:
        try:
            if need_api:
                relation = api.query("""
                relation(%s);
                (._;>;);
                out;
                """ % relation.id).relations[0]
            break
        except Exception as e:
            print e
            time.sleep(10)
    return relation

def way2line(way,need_api=True):
    while True:
        try:
            if need_api:
                way = api.query("""
                way(%s);
                (._;>;);
                out;
                """% way.id).ways[0]
            break
        except Exception as e:
            print e
            time.sleep(10)
    coors = lonlat_in_way(way)
    return shpgeo.LineString(coors)

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    from math import radians, cos, sin, asin, sqrt
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    m = km *1000
    return m
       
def handle_poly(name_pattern,poly):
    ext_coords = poly.exterior.coords
    centr = poly.centroid
    radius = get_max_dis_from_center_to_ext(centr, ext_coords)
    center_str = '%.6f,%.6f' %(centr.coords[0][0], centr.coords[0][1])
    return '%s\t%s\t%s\t%s\n' %(name_pattern, center_str, radius, list(poly.exterior.coords))
    

In [105]:
def remove_equal_shpobj(objs):
    import rtree
    size = len(objs)
    equal_pair = []
    keep = []
    exclude_idx = set()

    tree_idx = rtree.index.Index()
    objs_bounds = [o.bounds for o in objs]
    for i in xrange(size):
        try:
            tree_idx.insert(i, objs_bounds[i])
        except Exception as e:
            print i, objs_bounds[i], objs[i]
            raise e

    for i in xrange(size):
        if i in exclude_idx:
            continue
        keep.append(i)
        js = tree_idx.intersection(objs[i].bounds)
        for j in js:
            if i!=j and objs[i].equals(objs[j]):
                equal_pair.append((i,int(j)))
                exclude_idx.add(j)

    return keep, equal_pair
def remove_equal_shpobj(objs):
    import rtree
    size = len(objs)
    equal_pair = []
    keep = []
    exclude_idx = set()

    tree_idx = rtree.index.Index()
    objs_bounds = [o.bounds for o in objs]
    for i in xrange(size):
        try:
            tree_idx.insert(i, objs_bounds[i])
        except Exception as e:
            print i, objs_bounds[i], objs[i]
            raise e

    for i in xrange(size):
        if i in exclude_idx:
            continue
        keep.append(i)
        js = tree_idx.intersection(objs[i].bounds)
        for j in js:
            if i!=j and objs[i].equals(objs[j]):
                equal_pair.append((i,int(j)))
                exclude_idx.add(j)

    return keep, equal_pair
def rltn2dictShp(relation, sub_rltn=False):
    nodes, ways, sub_nodes, sub_ways = [], [], [], []
    for m in relation.members:
        obj = m.resolve()
        if isinstance(obj, overpy.Node):
            nodes.append(obj)
        elif isinstance(obj, overpy.Way):
            ways.append(obj)
        elif isinstance(obj, overpy.Relation):
            r_nodes, r_ways = rltn2dictShp(osm_container, obj, True)
            sub_nodes.extend(r_nodes)
            sub_ways.extend(r_ways)
    if sub_rltn:
        return nodes, ways
    nodes.extend([node for node in sub_nodes])
    ways.extend([way for way in sub_ways])

    points = [node2pt(node) for node in nodes]
    keep_pts_idx, _ = remove_equal_shpobj(points)
    points = [p for cnt, p in enumerate(points) if cnt in keep_pts_idx]

    lines = [way2line(way,False) for way in ways]
    keep_lines_idx, _ = remove_equal_shpobj(lines)
    lines = [l for cnt, l in enumerate(lines) if cnt in keep_lines_idx]

    dict_shp= {'Point': points, 'LineString': [], 'Polygon': []}
    if lines:
        merged = linemerge(lines)
        if merged.type == 'LineString':
            merged = [merged]
        else:
            merged = list(merged)
        for ln in merged:
            if ln.is_ring:
                dict_shp['Polygon'].append(shpgeo.Polygon(ln))
            else:
                dict_shp['LineString'].append(ln)
    return dict_shp
def merge_within_by_list_shp(list_shp):
    import geopandas as gp
    import pandas as pd
    from other_utils import find_tree
    gpdf = gp.GeoDataFrame(list_shp,columns=['geometry'])
    sjoin = gp.tools.sjoin(gpdf,gpdf,op='within')
    messy_tree_df = pd.DataFrame(zip(sjoin.index.values, sjoin.index_right.values), columns=['node','parent'])
    clean_tree_df = find_tree(messy_tree_df)
    top_level_shp_idx = clean_tree_df[clean_tree_df.parent==-1].node.values
    return gpdf[gpdf.index.isin(top_level_shp_idx)].values

def rltn2mergedListShp( relation):
    shpcltn = rltn2dictShp(relation)
    list_shp = []
    for l in shpcltn.values():
        list_shp += l
    merge_list_shp = merge_within_by_list_shp(list_shp)
    return merge_list_shp


def rltn2mergedFlattenListShp(relation):
    merge_list_shp = rltn2mergedListShp(relation)
    flat_shpcltn = []
    for shpobjs in merge_list_shp:
        flat_shpcltn.extend(shpobjs)
    return flat_shpcltn

def rltn2convexhull(relation):
    return shpgeo.MultiPolygon(rltn2mergedFlattenListShp(relation)).convex_hull

In [89]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../utils'))


In [110]:

def get_max_dis_from_center_to_ext(centr,ext_coords):
    lon1, lat1 = centr
    return max([haversine(lon1,lat1, lon2,lat2) for lon2, lat2 in ext_coords])

def get_cntr_radius(poly):
    cntr = poly.centroid.coords[0]
    ext_coords = poly.exterior.coords
    radius = get_max_dis_from_center_to_ext(cntr, ext_coords)
    return cntr, radius, ext_coords

def osm2polys(osm_data):
    if isinstance(osm_data, Way):
        ln = way2line(osm_data)
        poly = shpgeo.Polygon(ln)
        return [poly]
    if isinstance(osm_data, Relation):
        r = get_relation(osm_data)
        return rltn2mergedFlattenListShp(r)

def osms2flattenPolys(data_file_dir, geojson_file):
    place_polys = []
    for i, file_path in enumerate(glob.glob(data_file_dir+'/*')):
#         if i<=0:
#             continue
#         if i >5:
#             break
        _, place = file_path.split('\\')
        place = place[:-4].replace(' ','_')
        osm_data = parse_file(file_path)
        osm_data = list(osm_data)
        assert len(osm_data)==1, '{}th {}: len!=0'.format(i, file_path)
        print i, place, len(list(osm_data))
        osm_data = osm_data[0]
        polys = osm2polys(osm_data)
        for cnt, poly in enumerate(polys):
            cntr, radius, ext_coords = get_cntr_radius(poly)
            place_polys.append(['{}##{}'.format(place,cnt), cntr, radius+1000, poly])
        
    print len(place_polys)
    return place_polys


In [17]:

OSM_XML_DIR = u'../data/OSM_national_park@US/'
PLACE_POLYS_NP = '../data/place_polys_np.geojson'

In [153]:
place_polys = osms2flattenPolys(OSM_XML_DIR,PLACE_POLYS_NP)

0 Acadia_NP_(ACAD) 1
1 Adams_NHP 1
2 African_Burial_Ground_NM_(AFBG) 1
3 Agate_Fossil_Beds_NM 1
4 Alibates_Flint_Quarries_NM_(ALFL) 1
5 Allegheny_Portage_Railroad_NHS_(ALPO) 1
6 Andersonville_NHS_(ANDE) 1
7 Aniakchak_NM_&_PRES_(ANIA)1 1
8 Aniakchak_NM_&_PRES_(ANIA)2 1
9 Antietam_NB_(ANTI) 1
10 Appomattox_Court_House_NHP_(APCO) 1
11 Arches_NP_(ARCH) 1
12 Arkansas_Post_NMEM_(ARPO) 1
13 Arlington_House_The_R.E._Lee_MEM_(ARHO) 1
14 Assateague_Island_NS_(ASIS) 1
15 Aztec_Ruins_NM_(AZRU) 1
16 Badlands_NP_(BADL) 1
17 Bandelier_NM_(BAND) 1
18 Bent's_Old_Fort_NHS_(BEOL) 1
19 Bering_Land_Bridge_NPRES_(BELA) 1
20 Big_Bend_NP_(BIBE) 1
21 Big_Cypress_NPRES_(BICY) 1
22 Big_Hole_NB_(BIHO) 1
23 Big_South_Fork_NRRA_(BISO) 1
24 Bighorn_Canyon_NRA 1
25 Biscayne_NP 1
26 Black_Canyon_of_the_Gunnison_NP 1
27 Blue_Ridge_PKWY 1
28 Brown_v._Board_of_Education_NHS_[larger] 1
29 Brown_v._Board_of_Education_NHS 1
30 Bryce_Canyon_NP 1
31 Buck_Island_Reef_NM 1
32 Buffalo_NR 1
33 Cabrillo_NM 1
34 Canyon_de_Chelly_NM

In [171]:
gpdf = gp.GeoDataFrame(place_polys,columns=['place##cnt', 'cntr','radius+1km','geometry'])
gpdf.cntr = gpdf.cntr.apply(str)

In [172]:
with open(PLACE_POLYS_NP,'w') as f:
    f.write(gpdf.to_json())

In [160]:
gpdf[gpdf['place##cnt'].str.contains('Blue_Ridge_PKWY')]

,place##cnt,cntr,radius+1km,geometry,id
69,Blue_Ridge_PKWY##0,"(-81.6609656393, 36.1282316748)",168168.707947,POLYGON ((-83.38083338844375 35.55018165724734...,1
70,Blue_Ridge_PKWY##1,"(-79.7625342719, 37.3579361787)",42196.275031,"POLYGON ((-80.2247187355749 37.13233249453511,...",1
71,Blue_Ridge_PKWY##2,"(-79.1691825763, 37.7864583596)",41560.538870,POLYGON ((-79.60002433837357 37.53270358645526...,1


In [158]:
import geopandas as gp
gpdf.geometry = gpdf.buffer(0.1)
gpdf['id'] = 1
with open('nationalpark_polygons.js','w') as f:
    f.write('var np = %s;'%gpdf.to_json())

In [5]:
with open('nationalpark_boundbox_osm.tsv','w') as f:
    for i, poly in enumerate(polys):
        name_pattern = national_parks[i]
        bbox = poly.bounds
        bbox_str = ','.join([str(x)for x in bbox])  
        center_str = '%.6f,%.6f' %(poly.centroid.coords[0][0], poly.centroid.coords[0][1])
        f.write('%s\t%s\t%s\n' %(name_pattern, center_str, bbox_str))
        

unicode

In [178]:
type(place_gpdf.cntr.iloc[0])

tuple

In [187]:
place_gpdf = gp.read_file(PLACE_POLYS_NP)
radius = place_gpdf['radius+1km'].apply(lambda x: '{}km'.format(int(x/1000)+1)).values
cntr = place_gpdf.cntr.apply(eval).apply(lambda x: (x[1],x[0]))
place = place_gpdf['place##cnt'].values
PLACES = zip(place, cntr, radius)

In [188]:
PLACES


[(u'Acadia_NP_(ACAD)##0', (44.03591693771104, -68.63594571491548), '5km'),
 (u'Acadia_NP_(ACAD)##1', (44.071979451544216, -68.61921256814911), '2km'),
 (u'Acadia_NP_(ACAD)##2', (44.01569898061631, -68.61466787965567), '2km'),
 (u'Acadia_NP_(ACAD)##3', (44.06216793649834, -68.62017908924948), '2km'),
 (u'Acadia_NP_(ACAD)##4', (44.04258626567006, -68.5571333434889), '2km'),
 (u'Acadia_NP_(ACAD)##5', (44.04757410490562, -68.5554604616539), '2km'),
 (u'Acadia_NP_(ACAD)##6', (44.10061590915099, -68.47392536170815), '2km'),
 (u'Acadia_NP_(ACAD)##7', (44.289479952235446, -68.40092567665117), '2km'),
 (u'Acadia_NP_(ACAD)##8', (44.2834198591617, -68.39625072474915), '2km'),
 (u'Acadia_NP_(ACAD)##9', (44.42417457132982, -68.36145367515483), '2km'),
 (u'Acadia_NP_(ACAD)##10', (44.3928257854742, -68.35413223291408), '2km'),
 (u'Acadia_NP_(ACAD)##11', (44.397895299392715, -68.3506312550204), '2km'),
 (u'Acadia_NP_(ACAD)##12', (44.253005260814014, -68.34863353489996), '2km'),
 (u'Acadia_NP_(ACAD)##1